In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv("flights.csv")
df.head()

C:\Users\hovha\AppData\Local\Temp\ipykernel_10312\4204086180.py:5: DtypeWarning:

Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.



,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
delay_cols = ['ARRIVAL_DELAY', 'DEPARTURE_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df[delay_cols] = df[delay_cols].fillna(0)

df = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]

In [24]:
avg_delay_airline = df.groupby('AIRLINE')['ARRIVAL_DELAY'].mean().sort_values(ascending=False).reset_index()

fig = px.bar(avg_delay_airline, x='AIRLINE', y='ARRIVAL_DELAY',
             title='Average Arrival Delay per Airline',
             labels={'ARRIVAL_DELAY': 'Average Delay (min)', 'AIRLINE': 'Airline'})

fig.show()

This bar chart shows the average arrival delay per airline. Airlines with higher bars consistently arrive late, indicating possible scheduling or operational inefficiencies.

In [25]:
df['SCHEDULED_DEPARTURE_HOUR'] = df['SCHEDULED_DEPARTURE'] // 100

hourly_delay = df.groupby('SCHEDULED_DEPARTURE_HOUR')['ARRIVAL_DELAY'].mean().reset_index()

fig = px.line(hourly_delay, x='SCHEDULED_DEPARTURE_HOUR', y='ARRIVAL_DELAY',
              title='Average Arrival Delay by Scheduled Departure Hour',
              labels={'SCHEDULED_DEPARTURE_HOUR': 'Scheduled Hour', 'ARRIVAL_DELAY': 'Avg Arrival Delay (min)'})

fig.show()

Flights later in the day tend to be more delayed. This could be due to cumulative delays throughout the day, suggesting morning flights are more punctual.

In [26]:
top_routes = df.groupby(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']).size().reset_index(name='flight_count')
top_routes = top_routes.sort_values('flight_count', ascending=False).head(10)

fig = px.bar(top_routes, x='flight_count', y=top_routes['ORIGIN_AIRPORT'] + " → " + top_routes['DESTINATION_AIRPORT'],
             orientation='h',
             title='Top 10 Busiest Routes',
             labels={'flight_count': 'Number of Flights'})

fig.show()

These routes represent the most traveled city pairs in the dataset. They could indicate major hubs or popular corridors.

In [27]:
causes = delay_cols[2:]
cause_sums = df[causes].sum().sort_values(ascending=False)

fig = px.bar(cause_sums, x=cause_sums.index, y=cause_sums.values,
             title='Total Delay Minutes by Cause',
             labels={'x': 'Cause', 'y': 'Total Delay Minutes'})

fig.show()

The largest contributor to delays is usually late aircraft, followed by airline-related or weather-related causes.

In [28]:
# Create DataFrame with IATA, latitude, longitude
airport_coords = pd.DataFrame({
    'IATA_CODE': [
        'ATL', 'LAX', 'ORD', 'DFW', 'DEN', 'JFK', 'SFO', 'SEA', 'LAS', 'MCO',
        'CLT', 'PHX', 'IAH', 'MIA', 'BOS', 'MSP', 'FLL', 'DTW', 'PHL', 'BWI',
        'SLC', 'SAN', 'TPA', 'HNL', 'BNA', 'AUS', 'DAL', 'MDW', 'DCA', 'STL',
        'OAK', 'PDX', 'SJC', 'MSY', 'RDU', 'SMF', 'CLE', 'SAT', 'IND', 'CMH',
        'SNA', 'RSW', 'MKE', 'OMA', 'BUR', 'ONT', 'LGA', 'ELP', 'OKC', 'TUL'
    ],
    'LATITUDE': [
        33.6407, 33.9416, 41.9742, 32.8998, 39.8617, 40.6413, 37.6213, 47.4502, 36.0840, 28.4294,
        35.2140, 33.4342, 29.9902, 25.7959, 42.3656, 44.8820, 26.0726, 42.2162, 39.8744, 39.1754,
        40.7899, 32.7338, 27.9755, 21.3187, 36.1245, 30.1945, 32.8471, 41.7868, 38.8512, 38.7487,
        37.7126, 45.5898, 37.3639, 29.9934, 35.8776, 38.6954, 41.4117, 29.5337, 39.7173, 39.9980,
        33.6757, 26.5362, 42.9472, 41.3032, 34.2007, 34.0559, 40.7769, 31.8072, 35.3931, 36.1984
    ],
    'LONGITUDE': [
        -84.4277, -118.4085, -87.9073, -97.0403, -104.6731, -73.7781, -122.3790, -122.3088, -115.1537, -81.3089,
        -80.9431, -112.0116, -95.3368, -80.2870, -71.0096, -93.2218, -80.1527, -83.3554, -75.2424, -76.6684,
        -111.9791, -117.1907, -82.5333, -157.9224, -86.6782, -97.6699, -96.8518, -87.7524, -77.0402, -90.3700,
        -122.2197, -122.5951, -121.9289, -90.2580, -78.7875, -121.5916, -81.8498, -98.4698, -86.2944, -82.8919,
        -117.8678, -81.7552, -87.8966, -95.8997, -118.3576, -117.6005, -73.8740, -106.3777, -97.6005, -95.8881
    ]
})

In [29]:
merged = route_counts.merge(airport_coords, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='left')
merged = merged.rename(columns={'LATITUDE': 'origin_lat', 'LONGITUDE': 'origin_lon'})
merged.drop(columns=['IATA_CODE'], inplace=True)

merged = merged.merge(airport_coords, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='left')
merged = merged.rename(columns={'LATITUDE': 'dest_lat', 'LONGITUDE': 'dest_lon'})
merged.drop(columns=['IATA_CODE'], inplace=True)

merged = merged.dropna(subset=['origin_lat', 'origin_lon', 'dest_lat', 'dest_lon'])

fig = go.Figure()

for _, row in merged.iterrows():
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[row['origin_lon'], row['dest_lon']],
        lat=[row['origin_lat'], row['dest_lat']],
        mode='lines',
        line=dict(width=1, color='blue'),
        opacity=row['count'] / merged['count'].max()
    ))

fig.update_layout(
    title_text='Most Frequent U.S. Flight Routes',
    showlegend=False,
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        subunitcolor='rgb(217, 217, 217)',
        countrycolor='rgb(217, 217, 217)',
    )
)

fig.show()

The map shows the most frequent U.S. Flight Routes (with available location data), revealing strong air traffic concentration between major hubs like LAX, ATL, and ORD. It highlights the backbone of the national air travel network.